# F.B.A. : Analysis of the flow of metabolites through our metabolic network 

# Goal : Assessing the impact of at least one modification in the inputs and the outputs of the metabolic network (Blätke and Bräutigam, 2019) 

**F.B.A.** is a mathematical approach for analyzing the flow of metabolites through a metabolic network.
In F.B.A., we compute the steayd-state rate "fluxes" for each reaction in a given metabolic network. These are those fluxes which are mapped into the map of the metabolic network in the previous section.
The metabolic map is basically the visualization of the metabolic network that is encoded in the SBML file.

**Two major steps**:
- Define the constraints and represent metabolic reactions
- Define a phenotype in the form of a biological object that is relevant to the problem being studied


**Goal : Assessing the impact of a modification in the inputs of the metabolic network in the outputs** 
(Blätke and Bräutigam, 2019)


**Inputs :** 

- carbon dioxide
- light
- nitrogen
- ammonia
- phosphate
- sulfate


**Outputs :**

- starch
- sucrose
- amino acids

# Effect of CO2 rate uptake on C3 metabolism

# 0 / Initialization 

## 0.1 / Load packages 

In [1]:
# Import sys
import sys
sys.path.append("../src/") # add a specific path for interpreter to search => import all files located in src folder
sys.path.append("../src/init_fba") # to import all the content of the init_fba folder

In [2]:
# Import init for initialization and load user-defined functions
from init_fba import *

In [3]:
import plotly.graph_objects as go

In [4]:
import plotly.express as px

In [5]:
# Import notebook and initialize notebook settings
theNotebook = '2019-05-06-mb-genC3-CO2-Effect'
init_notebook(theNotebook)

In [6]:
#load sbml model
c3_model = load_sbml_model()
c3_model

Name,c3_model
Memory address,0x07ffb144ddee0
Number of metabolites,413
Number of reactions,572
Number of groups,0
Objective expression,1.0*Ex_Suc - 1.0*Ex_Suc_reverse_fb96e
Compartments,"Chloroplast, Lumen, Cytosol, Mitochondrion, IntermembraneSpace, Peroxisome"


In [7]:
#infinite variable defined at 1e6  
inf = float(1e6) 

# I / Metabolites models 

## I.1 / Constraints 

In [8]:
#CONSTRAINT: Set flux of all export reaction to zero => Steady-state
for r_obj in c3_model.reactions: # for loop to iterate on all the reactions in the C3 model
    r_id = r_obj.id
    if r_id[0:2] == "Ex": # if the reaction starts by "Ex" on the 2 first characters ([0:2]), we assign the limit (0., 0.)
        r_obj.bounds = (0.,0.)

In [9]:
#CONSTRAINT: Divergent fluxes of export and import reactions => Assigne the limits of all these following reactions
# Fix the limits of the flux : inputs and outputs
set_bounds('Im_CO2', (-inf, inf), c3_model) # input
set_bounds('Im_H2O', (-inf, inf), c3_model) # input
set_bounds('Im_H2S', (0.,0.), c3_model) # input
set_bounds('Im_NH4', (0., 0.), c3_model) # input
set_bounds('Im_NO3', (0., inf), c3_model) # input
set_bounds('Im_Pi', (0., inf), c3_model) # input
set_bounds('Im_SO4', (0., inf), c3_model) # input
set_bounds('Ex_O2', (-inf, inf), c3_model) # output
set_bounds('Ex_Suc', (0., inf), c3_model) # output
set_bounds('Ex_starch', (0., inf), c3_model) # output
set_bounds('Ex_AA', (0., inf), c3_model) # output
# => Initial conditions of the experiment

In [10]:
#CONSTRAINT: blocking these 2 following reactions of the plants in C3, according to the specie considered 
set_bounds('G6PDH_h', (0.,0.), c3_model) # h : chloroplast
set_bounds('PPIF6PK_c', (0,0.), c3_model) # c : cytosol
# => Initial conditions of the specie of interest

In [11]:
#CONSTRAINT: max. photon consumption 1000 μE
set_bounds('Im_hnu', (0, 1000), c3_model)
# => Initial conditions concerning light, received photons

In [12]:
#CONSTRAINT: CO2 uptake rate in C3 plants is about 20 μmol/(m2*s)
f_CO2 = 20 #[μmol/(m2*s)] 
set_bounds('Im_CO2', (0, f_CO2), c3_model)
# => Define again the initial condition of CO2 input
# => Maybe different simulations with different values of CO2

In [13]:
#CONSTRAINT: Maintenance cost 

atp_cost_L3_m = 0.009111187245501572 #Mitochondria-L3-ATP Cost [µmol*s-1*m-2] ; L3 = third larval stage : mature Arabidopsis
atp_cost_L3_h = 0.15270708327974447 #Chloroplast-L3-ATP Cost [µmol*s-1*m-2]
atp_cost_L3_p = 0.0076669066992201855 #Peroxisome-L3-ATP Cost [µmol*s-1*m-2]
atp_cost_L3_c = 0.042683072918274702 #Cytosl/Other-L3-ATP Cost [µmol*s-1*m-2]

set_fixed_flux('NGAM_c',atp_cost_L3_c + atp_cost_L3_p, c3_model) # flux is fixed with a cost ; NGAM = maintenance reaction
set_fixed_flux('NGAM_m',atp_cost_L3_m, c3_model)
set_fixed_flux('NGAM_h',atp_cost_L3_h, c3_model)
# => Consumption of ATP for a mature leaf of Arabidopsis thaliana
# Fix the flux of the maintenance reaction for the 3 compartments
# What's the maintenance reaction? ATP_{x} + H2O_{x} = ADP{x} + H_{x} + Pi_{x}, x = c;h;m
# Why the addition? See next paragraph in the paper : 
# "The peroxisomal maintenance costs are added to the cytosolic maintenance costs."

In [14]:
#CONSTRAINT: Output of sucrose : total amino acid and sucrose : starch
set_fixed_flux_ratio({'Ex_Suc':2.2,'Ex_AA':1.0}, c3_model)
set_fixed_flux_ratio({'Ex_Suc':1.0,'Ex_starch':1.0}, c3_model)
# => Definition of the constraints, according to the ratio of sucrose and amino acid on the one hand, the ration of sucrose and
# starch on the other hand

In [15]:
#CONSTRAINT: oxygenation : decarboxylation = 1 : 10
set_fixed_flux_ratio({'RBC_h':10,'RBO_h':1}, c3_model)
# => Definition of the constraints for the RubisCO, according to its ratio of oxygenation / decarboxylation states

In [16]:
#CONSTRAINT: fluxes through the chloroplastic NADPH dehydrogenase and plastoquinol oxidase were set to zero 
#because the contributions of NADPH dehydrogenase (Yamamoto et al., 2011) and plastoquinol oxidase 
#(Josse et al., 2000) to photosynthesis are thought to be minor.
set_bounds('AOX4_h',(0,0), c3_model)
set_bounds('iCitDHNADP_h',(0,0), c3_model)
# => Definition of constraints for reactions which seem to be minor (fluxes through the chloroplastic NADPH dehydrogenase and 
# plastoquinol oxidase here)

In [17]:
#CONSTRAINT: NTT is only active at night; NTT = nitrate transporter/transferase?
set_fixed_flux('Tr_NTT',0, c3_model)

In [18]:
#CONSTRAINT: No uncoupled pyruvate transport
set_bounds('Tr_Pyr1',(0,0), c3_model)
set_bounds('Tr_Pyr2',(0,0), c3_model)

# II / FBA

## II.1 / Set FBA solver  

In [19]:
#Set FBA solver
c3_model.solver = "glpk" # GLPK = Gnu Linear Porgramming Kit
# => Definition of the solver

## II.2 / Optimize/Maximize the output 

In [20]:
#Optimize/Maximize sucrose output
result_fba_c3 = c3_model.optimize('maximize') #perform FBA
Ex_Suc = c3_model.reactions.get_by_id("Ex_Suc")
Ex_Suc.objective_coefficient = 1.

## II.3 / Optimize/Minimize total flux 

In [21]:
#Optimize/Minimize total flux
if result_fba_c3.status == 'optimal': 
    result_pfba_c3 = cobra.flux_analysis.parsimonious.pfba(c3_model)
result_pfba_c3

,fluxes,reduced_costs
PSII_h,24.213901,-2.000000
Cytb6f1_h,48.427803,-2.000000
Cytb6f2_h,48.427803,-2.000000
PGR5PGRL11_h,0.000000,5.168224
PGR5PGRL12_h,0.000000,-2.000000
...,...,...
Tr_KG_Mal_mc,-0.086595,2.000000
Tr_Asp_mc,0.161108,-2.000000
Tr_Asp_Glu_mc,0.000000,2.000000
Tr_Pyr_Mal_hc,0.049169,-2.000000


## II.4 / Fetch flux from CO2 uptake 

In [22]:
#Fetch flux for CO2 uptake
flux_CO2 = result_pfba_c3.fluxes['Im_CO2']
flux_CO2

19.999999999999957

In [23]:
#Array defining proprtion of CO2 uptake 
L_CO2 = np.linspace(0,1,21) # np.linspace() => Return evenly spaced numbers over a specified interval.
L_CO2

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ])

## II.5 / Store results 

In [24]:
#Initialize dictionary to store results
D_fba = {}

In [25]:
#Iterate over proportions of CO2 uptake
for v_co2 in L_CO2:
    
    #Fix upper flux bound for photon uptake
    set_bounds('Im_CO2', (0, flux_CO2 * v_co2), c3_model)
    
    #Optimize/Maximize sucrose output
    result_fba_c3 = c3_model.optimize('maximize') #perform FBA
    
    #Optimize/Minimize total flux
    if result_fba_c3.status == 'optimal': # check if feasible
        result_pfba_c3  = cobra.flux_analysis.parsimonious.pfba(c3_model) #perform pFBA
        D_fba[flux_CO2 * v_co2] = result_pfba_c3.fluxes
    else:
        D_fba[flux_CO2 * v_co2] = pd.Series(pd.Series( index=[r_obj.id for r_obj in c3_model.reactions], data = [0]*len(c3_model.reactions)))

# III / Figures 

In [26]:
# Make plots with plotly => Scatter plots
xaxis_title = 'CO2 Uptake [µmol/s/m2]' # title of the x axis
save_fig = False # Static image

In [27]:
L_r = ['Ex_Suc','Ex_AA']
create_scatter_plot_rxn_c3(D_fba, L_r, 'Phloem Export', xaxis_title, save_fig = save_fig)

In [28]:
L_r = ['PSI_h','PSII_h']
create_bar_plot_met_c3(c3_model, D_fba, L_r, 'hnu_h', 'Photon Uptake by Photosystems', xaxis_title, save_fig = save_fig)

In [29]:
L_r = ['NDH1_h','NDH2_h']
create_scatter_plot_rxn_c3(D_fba, L_r, 'Cyclic Electron Flow', xaxis_title, save_fig = save_fig)

In [ ]:
L_r = ['PSI_h','PSII_h']
data_photosystem = px.D_fba.gapminder().query()